### Install Flask package using pip
The exclamation mark '!' is used to execute Linux commands. 
If the Flask package is installed, you can ignore the cell below.

In [136]:
#!pip3 install flask

In [137]:
#!pip install --upgrade flask


In [138]:
#!pip install watchdog

In [139]:
#!pip install ailabs_asr

In [140]:
#  環境依賴
import os
import gc
import sqlite3
import uuid
from datetime import datetime
from werkzeug.utils import secure_filename
from pydub import AudioSegment
from ailabs_asr.streaming import StreamingClient 
from flask import Flask, render_template, request, jsonify

In [141]:
app = Flask(__name__)

In [142]:
# 資料路徑設定
DATABASE_FOLDER = os.path.join(app.root_path, 'instance')
UPLOAD_FOLDER = os.path.join(app.root_path, 'uploads')

DATABASE = os.path.join(DATABASE_FOLDER, 'database.db')

In [143]:
# 設定允許的檔案類型
ALLOWED_EXTENSIONS = {'wav', 'mp3', 'm4a'}

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

In [144]:
# 確保所有需要的文件夾存在
for folder in [DATABASE_FOLDER]:
    if not os.path.exists(folder):
        os.makedirs(folder)

In [145]:
# 定義根目錄
@app.route('/')
def index():
    return render_template('index.html')

In [146]:
# 定義分類目錄入口
@app.route('/classify')
def classify():
    return render_template('class.html')

In [147]:
#定義身體頁面
@app.route('/body')
def bodyPage():
   return render_template('classTemplate.html', title = "BODY")

In [148]:
#定義心靈頁面
@app.route('/psycho')
def psychoPage():
   return render_template('classTemplate.html', title = "PSYCHO")

In [149]:
#定義社會頁面
@app.route('/social')
def socialPage():
   return render_template('classTemplate.html', title = "SOCIAL")

In [150]:
#定義特殊頁面
@app.route('/special')
def specialPage():
   return render_template('classTemplate.html', title = "SPECIAL")

In [151]:
#定義其他頁面
@app.route('/extra')
def extraPage():
   return render_template('classTemplate.html', title = "EXTRA")

In [152]:
# 查詢姓名列表
@app.route('/fetchNameList', methods=['GET']) 
def fetchNameList():
    try:
        # 使用 with 語句管理資料庫連線
        with sqlite3.connect(DATABASE) as conn:
            cursor = conn.cursor()
            cursor.execute("SELECT name FROM nameList")  # 假設表格名稱為 'nameList'，字段名稱為 'name'
            names = [row[0] for row in cursor.fetchall()]
        return jsonify(names)

    except Exception as e:
        return jsonify({'error': str(e)}), 500


In [153]:
# 新增姓名到資料庫
@app.route('/addName', methods=['POST'])
def addName():
    try:
        data = request.get_json()
        new_name = data.get('name')
        if not new_name:
            return jsonify({'error': 'No name provided'}), 400
        
        conn = sqlite3.connect(DATABASE)
        cursor = conn.cursor()
        cursor.execute("INSERT INTO nameList (name) VALUES (?)", (new_name,))
        conn.commit()
        conn.close()
        return jsonify({'success': True}), 200
    except Exception as e:
        return jsonify({'error': str(e)}), 500

In [154]:
# 查詢指定人名的逐字稿
@app.route('/fetchTranscripts', methods=['GET'])
def fetchTranscripts():
    person = request.args.get('person')  # 從請求中獲取人名參數
    if not person:
        return jsonify({'error': 'No person provided'}), 400

    try:
        conn = sqlite3.connect(DATABASE)
        cursor = conn.cursor()
        # 根據名稱查詢逐字稿
        cursor.execute("SELECT content, timestamp FROM transcripts WHERE name = ? ORDER BY timestamp DESC", (person,))
        rows = cursor.fetchall()
        transcripts = [{'content': row[0], 'timestamp': row[1]} for row in rows]
        conn.close()
        app.logger.info(f'Transcripts fetched for {person}: {transcripts}')

        return jsonify(transcripts)

    except Exception as e:
        return jsonify({'error': str(e)}), 500

In [155]:
# 新增指定人名的逐字稿
@app.route('/uploadTranscript', methods=['POST'])
def uploadTranscript():
    data = request.get_json()
    name = data.get('name')
    content = data.get('content')
    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    if not name or not content:
        app.logger.error('Name or content is missing in the request')
        return jsonify({'error': 'Name or content is missing'}), 400

    try:
        conn = sqlite3.connect(DATABASE)
        cursor = conn.cursor()
        cursor.execute("INSERT INTO transcripts (name, content, timestamp) VALUES (?, ?, ?)", (name, content, timestamp))
        conn.commit()
        conn.close()
        app.logger.info(f'Transcript uploaded for {name} at {timestamp}')
        return jsonify({'message': 'Transcript uploaded successfully'}), 200
    except Exception as e:
        app.logger.error(f'Error uploading transcript for {name}: {e}')
        return jsonify({'error': str(e)}), 500

In [156]:
# 編輯指定人名的逐字稿
@app.route('/editTranscript', methods=['POST'])
def editTranscript():
    data = request.get_json()
    timestamp = data.get('timestamp')
    new_content = data.get('newContent')

    if not timestamp or not new_content:
        app.logger.error('Timestamp or new content is missing in the request')
        return jsonify({'error': 'Timestamp or new content is missing'}), 400

    try:
        conn = sqlite3.connect(DATABASE)
        cursor = conn.cursor()
        cursor.execute("UPDATE transcripts SET content = ? WHERE timestamp = ?", (new_content, timestamp))
        conn.commit()
        conn.close()
        app.logger.info(f'Transcript edited at {timestamp}')
        return jsonify({'message': 'Transcript edited successfully'}), 200
    except Exception as e:
        app.logger.error(f'Error editing transcript at {timestamp}: {e}')
        return jsonify({'error': str(e)}), 500

In [157]:
# 刪除指定人名的逐字稿
@app.route('/deleteTranscript', methods=['POST'])
def deleteTranscript():
    data = request.get_json()
    timestamp = data.get('timestamp')

    if not timestamp:
        app.logger.error('Timestamp is missing in the request')
        return jsonify({'error': 'Timestamp is missing'}), 400

    try:
        conn = sqlite3.connect(DATABASE)
        cursor = conn.cursor()
        cursor.execute("DELETE FROM transcripts WHERE timestamp = ?", (timestamp,))
        conn.commit()
        conn.close()
        app.logger.info(f'Transcript deleted at {timestamp}')
        return jsonify({'message': 'Transcript deleted successfully'}), 200
    except Exception as e:
        app.logger.error(f'Error deleting transcript at {timestamp}: {e}')
        return jsonify({'error': str(e)}), 500

In [158]:
# 確保音頻格式
def ensure_audio_format(input_file_path, output_file_path):
    """
    將音頻文件轉換為指定的格式：
    16kHz, 單聲道, 16 bits per sample, PCM 格式
    """
    try:
        # 從文件中加載音頻
        audio = AudioSegment.from_file(input_file_path)

        # 設置音頻參數：16kHz 采樣率、單聲道、16位深度
        audio = audio.set_frame_rate(16000)
        audio = audio.set_channels(1)
        audio = audio.set_sample_width(2)  # 16 bits -> 2 bytes

        # 將文件導出為符合要求的 PCM WAV 文件
        audio.export(output_file_path, format="wav")
        
        print(f"音頻已成功轉換並保存至：{output_file_path}")
        
        # 顯式釋放音頻對象以確保文件不被鎖定
        del audio
        gc.collect()  # 手動進行垃圾回收，確保內存釋放
        
    except Exception as e:
        print(f"音頻格式處理失敗：{e}")
        raise

In [159]:
# 語音轉文字模塊
def transcribe_audio(file_path):
    """
    使用語音轉文字模塊來處理音頻文件，將音頻文件轉換為文本。
    """

    # 獲取金鑰檔案
    current_dir = os.getcwd()
    key_file_path = os.path.join(current_dir, 'key.txt')
    
    if not os.path.exists(key_file_path):
        raise FileNotFoundError(f"檔案 '{key_file_path}' 不存在，請檢查檔案路徑或內容！")

    # 繼續讀取金鑰
    with open(key_file_path, 'r') as file:
        api_key = file.read().strip()
        if api_key == "" or None:
            raise FileNotFoundError(f"金鑰為空！")

    transcript = []
    def on_processing_sentence(message):
        print(f'hello: {message["asr_sentence"]}')

    def on_final_sentence(message):
        transcript.append(message["asr_sentence"])
        #print(f'world: {message["asr_sentence"]}')
        
    asr_client = StreamingClient(key=api_key)

    # 開始語音轉文字處理
    asr_client.start_streaming_wav(
        pipeline='asr-zh-tw-std',
        file=file_path,
        #on_processing_sentence=on_processing_sentence,
        on_final_sentence=on_final_sentence
    )
    
    return transcript

In [160]:
# 將逐字稿保存到資料庫
def save_transcript_to_db(user, content):
    """
    將逐字稿保存到資料庫中。
    """
    try:
        conn = sqlite3.connect(DATABASE)
        cursor = conn.cursor()
        timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

        cursor.execute("INSERT INTO transcripts (name, content, timestamp) VALUES (?, ?, ?)", (user, content, timestamp))
        
        conn.commit()
        conn.close()
        print(f"逐字稿已成功儲存到資料庫，使用者：{user}")
    except Exception as e:
        app.logger.error(f'Error uploading transcript for {user}: {e}')

In [ ]:
# 上傳錄音檔
@app.route('/uploadRecord', methods=['POST'])
def upload_record():
    print("接收到上傳請求")
    if 'file' not in request.files:
        print("沒有檔案被上傳")
        return jsonify({'success': False, 'message': '沒有檔案被上傳'}), 400
    
    file = request.files['file']
    print(f"接收到的文件名稱：{file.filename}")

    if file.filename == '':
        print("文件名稱為空")
        return jsonify({'error': 'No selected file'}), 400

    # 檢查文件是否符合允許的類型
    if not allowed_file(file.filename):
        print(f"不允許的文件類型：{file.filename}")
        return jsonify({'error': 'File type not allowed'}), 400

    # 生成唯一文件名
    temp_file_path = os.path.join(UPLOAD_FOLDER, f"temp_{uuid.uuid4().hex}_{file.filename}")

    try:
        # 先儲存上傳的原始檔案到磁碟
        with open(temp_file_path, 'wb') as f:
            file.save(f)
        print(f"臨時文件已成功保存至：{temp_file_path}")

        # 轉換音頻格式並重新儲存，並刪除原始檔案
        converted_file_path = os.path.splitext(temp_file_path)[0] + "_converted.wav"
        ensure_audio_format(temp_file_path, converted_file_path)

        # 刪除原始文件
        if os.path.exists(temp_file_path):
            os.remove(temp_file_path)
        print(f"原始臨時文件已刪除：{temp_file_path}")

        print("正在轉換逐字稿......")
        contents = transcribe_audio(converted_file_path)
        print("轉換完成")
        print(contents)
        user = request.form.get('user')
        
        # 使用 join 生成合併後的內容
        transcript = "\n".join(contents)
        
        print(transcript)    
        save_transcript_to_db(user, transcript)
        print("逐字稿成功寫入資料庫")

    except Exception as e:
        print(f"音頻轉換失敗：{e}")
        # 如果轉換失敗，刪除臨時文件
        if os.path.exists(temp_file_path):
            os.remove(temp_file_path)
        return jsonify({'error': 'Audio conversion failed'}), 500

    return jsonify({'message': 'File uploaded and converted successfully', 'converted_file_path': converted_file_path}), 200

In [ ]:
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.2.104:5000
Press CTRL+C to quit
192.168.2.104 - - [28/Nov/2024 21:45:38] "GET / HTTP/1.1" 200 -
192.168.2.104 - - [28/Nov/2024 21:45:38] "GET /static/images/profile/01.png HTTP/1.1" 304 -
192.168.2.104 - - [28/Nov/2024 21:45:38] "GET /static/css/style.css HTTP/1.1" 304 -
192.168.2.104 - - [28/Nov/2024 21:45:38] "GET /static/js/index.js HTTP/1.1" 200 -
192.168.2.104 - - [28/Nov/2024 21:45:38] "GET /fetchTranscripts?person=張偉 HTTP/1.1" 200 -
192.168.2.104 - - [28/Nov/2024 21:45:38] "GET / HTTP/1.1" 200 -


接收到上傳請求
接收到的文件名稱：中台.wav
臨時文件已成功保存至：c:\Users\IOTLAB\Documents\GitHub\AI_Note_Service.github.io\APP1\uploads\temp_03f4bd5fc1314e77aeaa01bc63c996b4_中台.wav
音頻已成功轉換並保存至：c:\Users\IOTLAB\Documents\GitHub\AI_Note_Service.github.io\APP1\uploads\temp_03f4bd5fc1314e77aeaa01bc63c996b4_中台_converted.wav
原始臨時文件已刪除：c:\Users\IOTLAB\Documents\GitHub\AI_Note_Service.github.io\APP1\uploads\temp_03f4bd5fc1314e77aeaa01bc63c996b4_中台.wav
正在轉換逐字稿......
### start stream ###
### opened ###
{"status":"ok","ssid":"3f84b2a6-cdde-4020-881e-8ad81e99808c","message_type":"session_started"}
segment: 好現在我們要
segment: 好現在我們要進行一
segment: 好現在我們要進行一個簡單
segment: 好現在我們要進行一個簡單的
segment: 好現在我們要進行一個簡單的訪問
segment: 好現在我們要進行一個簡單的問你今天
segment: 好現在我們要進行一個簡單的問你今天晚上想
segment: 好現在我們要進行一個簡單的問你今天晚上想吃
segment: 好現在我們要進行一個簡單的問你今天晚上想吃什麼
segment: 好現在我們要進行一個簡單的問你今天晚上想吃什麼我今天要吃
segment: 好現在我們要進行一個簡單的問你今天晚上想吃什麼我今天要吃炒飯
segment: 那真是主要
segment: 那真是主要是我聽不
segment: 那真是主要是我聽不懂
segment: 那真是主要是我聽不懂台語
segment: 那真是主要是我聽不懂台語那你的外套是
segment: 那真是主要是我聽不懂台語那你的外套是哪裡

192.168.2.104 - - [28/Nov/2024 21:46:17] "POST /uploadRecord?name=張偉 HTTP/1.1" 200 -


### closed ###
Websocket closed[None]: None
轉換完成
['好，現在我們要進行一個簡單的問你，今天晚上想吃什麼，我今天要吃炒飯', '那真是，主要是我聽不懂台語，那你的外套是哪裡買的外套是在 uniqlo 買', '好的，我有聽懂 uniqlo 的部分，雖然這一段等一下翻得很奇怪，就這樣']
張偉 好，現在我們要進行一個簡單的問你，今天晚上想吃什麼，我今天要吃炒飯
那真是，主要是我聽不懂台語，那你的外套是哪裡買的外套是在 uniqlo 買
好的，我有聽懂 uniqlo 的部分，雖然這一段等一下翻得很奇怪，就這樣
逐字稿已成功儲存到資料庫，使用者：張偉
逐字稿成功寫入資料庫


192.168.2.104 - - [28/Nov/2024 21:46:19] "GET /fetchTranscripts?person=張偉 HTTP/1.1" 200 -
192.168.2.104 - - [28/Nov/2024 21:46:54] "POST /deleteTranscript HTTP/1.1" 200 -
192.168.2.104 - - [28/Nov/2024 21:46:56] "GET /fetchTranscripts?person=張偉 HTTP/1.1" 200 -
192.168.2.104 - - [28/Nov/2024 21:46:58] "POST /deleteTranscript HTTP/1.1" 200 -
192.168.2.104 - - [28/Nov/2024 21:46:59] "GET /fetchTranscripts?person=張偉 HTTP/1.1" 200 -
192.168.2.104 - - [28/Nov/2024 21:48:15] "POST /deleteTranscript HTTP/1.1" 200 -
192.168.2.104 - - [28/Nov/2024 21:48:16] "GET /fetchTranscripts?person=張偉 HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2024 21:49:04] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2024 21:49:57] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2024 21:49:57] "GET /static/css/style.css HTTP/1.1" 304 -
127.0.0.1 - - [28/Nov/2024 21:49:57] "GET /static/images/profile/01.png HTTP/1.1" 304 -
127.0.0.1 - - [28/Nov/2024 21:49:57] "GET /static/js/index.js HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2024 21:4

接收到上傳請求
接收到的文件名稱：中台2.wav
臨時文件已成功保存至：c:\Users\IOTLAB\Documents\GitHub\AI_Note_Service.github.io\APP1\uploads\temp_3b090c6ddc0b4f56bc2a615703ab479c_中台2.wav
音頻已成功轉換並保存至：c:\Users\IOTLAB\Documents\GitHub\AI_Note_Service.github.io\APP1\uploads\temp_3b090c6ddc0b4f56bc2a615703ab479c_中台2_converted.wav
原始臨時文件已刪除：c:\Users\IOTLAB\Documents\GitHub\AI_Note_Service.github.io\APP1\uploads\temp_3b090c6ddc0b4f56bc2a615703ab479c_中台2.wav
正在轉換逐字稿......
### start stream ###
### opened ###
{"status":"ok","ssid":"469bd402-98a7-449c-9dff-8760bc67c810","message_type":"session_started"}
segment: 好那我們
segment: 好那我們接下來
segment: 好那我們接下來進行第二
segment: 好那我們接下來進行第二次
segment: 好那我們接下來進行第二次測試我們
segment: 好那我們接下來進行第二次測試我們請開發
segment: 好那我們接下來進行第二次測試我們請開發人員
segment: 好那我們接下來進行第二次測試我們請開發人員來
segment: 好那我們接下來進行第二次測試我們請開發人員來講一下現在
segment: 好那我們接下來進行第二次測試我們請開發人員來講一下現在的狀況
segment: 好那我們接下來進行第二次測試我們請開發人員來講一下現在的狀況現在就是
segment: 好那我們接下來進行第二次測試我們請開發人員來講一下現在的狀況現在就是第一段的狀況
segment: 好那我們接下來進行第二次測試我們請開發人員來講一下現在的狀況現在就是第一段的狀況不錯
segment: 好那我們接下來進行

127.0.0.1 - - [28/Nov/2024 21:50:26] "POST /uploadRecord?name=張偉 HTTP/1.1" 200 -


### closed ###
Websocket closed[None]: None
轉換完成
['好，那我們接下來進行第二次測試，我們請開發人員來講一下，現在的狀況，現在就是第一段的狀況不錯，第二段還沒做好，', 'ok', '狀況就是這個狀況，謝謝']
張偉 好，那我們接下來進行第二次測試，我們請開發人員來講一下，現在的狀況，現在就是第一段的狀況不錯，第二段還沒做好，
ok
狀況就是這個狀況，謝謝
逐字稿已成功儲存到資料庫，使用者：張偉
逐字稿成功寫入資料庫


127.0.0.1 - - [28/Nov/2024 21:50:34] "GET /fetchTranscripts?person=張偉 HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2024 21:50:45] "GET /fetchNameList HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2024 21:50:46] "GET /fetchTranscripts?person=李麗芬 HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2024 21:50:49] "GET /fetchNameList HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2024 21:50:49] "GET /fetchTranscripts?person=張偉 HTTP/1.1" 200 -
